In [0]:
!pip install h2o
!pip install pandarallel --user
#from pandarallel import pandarallel
#pandarallel.initialize(progress_bar = True)

In [0]:
#google drive os
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("drive/My Drive/Colab Notebooks/Data/FeatureEngineering")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix
from termcolor import colored

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn import metrics
import xgboost as xgb


from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch

In [0]:
from h2o.automl import H2OAutoML
import h2o
from h2o.frame import H2OFrame
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,2 hours 29 mins
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.2
H2O cluster version age:,"7 days, 23 hours and 54 minutes"
H2O cluster name:,H2O_from_python_unknownUser_sax6iw
H2O cluster total nodes:,1
H2O cluster free memory:,2.569 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"


In [0]:
def df_to_h2oF(data, factor_list):
    
    data[factor_list] = data[factor_list].astype(int)
    h2o_df = H2OFrame(data)
    h2o_df[factor_list] = h2o_df[factor_list].asfactor()
    
    return h2o_df

In [0]:
def predict_es_with_saved_model(filename):
  
    data_es = pd.read_csv(filename, index_col= 0, parse_dates=[0])
    #need to change to leader_standardize_model
    
    #no standardize at all
    #standardized_saved_model = h2o.load_model('./leader_xgb_0423/XGBoost_1_AutoML_20190424_012109')
    
    #lianjinshushi
    standardized_saved_model = h2o.load_model('./buNeng_shuffle/XGBoost_1_AutoML_20190425_015417')
    
    #all standardize
    #tandardized_saved_model = h2o.load_model('./leader_standardize_0424/XGBoost_1_AutoML_20190424_070314') 
    
    #standardized_saved_model = h2o.load_model('./leader_standardize_model/XGBoost_1_AutoML_20190424_221520')
    
    remove_data =  data_es[~data_es.isin([np.nan, np.inf, -np.inf]).any(1)]
    factor_list_es = ['feature_57', 'feature_58', 'Feature_45','Feature_47', 'Target']
    numeric_list_es = list(set(remove_data.columns) - set(factor_list_es))
    
    remove_data[numeric_list_es] = remove_data[numeric_list_es].apply(lambda x: (x-np.mean(x))/np.std(x), axis = 0)
    
    #true label
    X = remove_data.iloc[:,:-1]
    y = remove_data['Target']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/2, shuffle = False)
    
    train_mu = X_train[numeric_list_es].mean()
    train_sd = X_train[numeric_list_es].std()
    
    #X_train[numeric_list_es] = (X_train[numeric_list_es] - train_mu) / train_sd
    
    test_mu = X_test[numeric_list_es].mean()
    test_sd = X_test[numeric_list_es].std()
    
    #test itself standardize:
    #X_test[numeric_list_es] = (X_test[numeric_list_es] - test_mu) / test_sd
    
    #test using train to stand
    #X_test[numeric_list_es] = (X_test[numeric_list_es] - train_mu) / train_sd
    
    #for Ren Hao's usage
    #pd.DataFrame(X_train.index).to_csv('cleaned_train_index_es.csv')
    #pd.DataFrame(X_test.index).to_csv('cleaned_test_index_es.csv')
    
 
    preds_standardize = standardized_saved_model.predict(df_to_h2oF(X_test, factor_list_es[:-1]))

    
    #standardized_es_predict_df = h2o.as_list(preds_standardize)
    #standardized_es_predict_df.to_csv('standardized_corn_predict.csv')
    
    cm = confusion_matrix(y_test.values, (h2o.as_list(preds_standardize)['predict']>0.5))
    
    
    print(colored('The Confusion Matrix is: ', 'red'),'\n', cm)
    predict_accuracy_on_test_set = (cm[0,0] + cm[1,1])/(cm[0,0] + cm[1,1]+cm[1,0] + cm[0,1])
    print(colored('The Accuracy on Test Set is: ', 'blue'), colored(predict_accuracy_on_test_set, 'blue'))
    print(metrics.classification_report(y_true=y_test.values,  y_pred=(h2o.as_list(preds_standardize)['predict']>0.5).astype(float)))

In [0]:
predict_es_with_saved_model('Report_Features.csv')

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%
The Confusion Matrix is:  
 [[11599    51]
 [   61    23]]
The Accuracy on Test Set is:  0.9904550877791034
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00     11650
         1.0       0.31      0.27      0.29        84

   micro avg       0.99      0.99      0.99     11734
   macro avg       0.65      0.63      0.64     11734
weighted avg       0.99      0.99      0.99     11734

